# 1. Introduction to Ranking Problem
Ranking documents against a query is one of the main problem tackled by Information Retrieval (IR).


## a) Information need, Query, Documents and Collections
Let's define the entities that are concerned with the ranking problem. A user (U) has an information need (I). He/She wants to identify a set of relevant documents (D) among a collection (C). Every documents is composed of a list of terms (T). The first step for the user is to translate his/her information need into a language that the system can understand, that is create a query. Such a query is often constituted of a list of keywords that the search system will try to match to the content of documents in the collection.

In [6]:
with open('./dataset/CISI.ALL') as f:
    lines = ""
    for l in f.readlines():
        lines += "\n" + l.strip() if l.startswith(".") else " " + l.strip()
    lines = lines.lstrip("\n").split("\n")
    
doc_set = {}
doc_id = ""
doc_text = ""
for l in lines:
    if l.startswith(".I"):
        doc_id = l.split(" ")[1].strip()
    elif l.startswith(".X"):
        doc_set[doc_id] = doc_text.lstrip(" ")
        doc_id = ""
        doc_text = ""
    else:
        doc_text += l.strip()[3:] + " " # The first 3 characters of a line can be ignored.

# Print something to see the dictionary structure
print(f"Number of documents = {len(doc_set)}" + ".\n")
print(doc_set["1"]) # note that the dictionary indexes are strings, not numbers. 

Number of documents = 1460.

18 Editions of the Dewey Decimal Classifications Comaromi, J.P. The present study is a history of the DEWEY Decimal Classification.  The first edition of the DDC was published in 1876, the eighteenth edition in 1971, and future editions will continue to appear as needed.  In spite of the DDC's long and healthy life, however, its full story has never been told.  There have been biographies of Dewey that briefly describe his system, but this is the first attempt to provide a detailed history of the work that more than any other has spurred the growth of librarianship in this country and abroad. 


## b) Evaluation measures (precision/recall/f1)
To evaluate the capacity of an Information Retrieval System, we need to be able to compute how well it can match documents against queries. The goal is not to make an absolute evaluation of the system performance but rather to be able to compare different systems. After a query, a set of matching documents is returned (*matching* according to the system of course). We want to evaluate how many of these documents really are relevant, and how many relevant documents were missed by the system. We often use the following terms when evaluating this.
- <u>True Positive (TP):</u> number of relevant documents returned by the system
- <u>False Positive (FP):</u> number of non-relevant documents returned by the system
- <u>True Negative (TN):</u> number of non-relevant documents **not** returned by the system
- <u>False Negative (FN):</u> number of relevant document **not** returned by the system.
### Precision
The precision is the capacity of a system to select <u>**only**</u> relevant documents towards a query.
If the system returns too many documents, of course a lot of them will match the user query. However, a lot of documents won't be interesting for the user and the precision will be low. On the contrary, if most of the returned documents are relevant, the precision will be high. 
$$Precision=\frac{TP}{TP+FP}$$
### Recall
The recall is the capacity of a system to select <u>**all**</u> relevant documents towards a query. If a lot of relevant documents are  not returned by the system, the recall will be low. On the contrary, if most relevant documents are returned by the system, the recall will be high.
$$Recall=\frac{TP}{TP+FN}$$
### F-score
Precision and recall measure different aspect of the search system. Both of them are important though. For example, it is very easy to get a high recall. If the system returns all documents in the collection, whether or not they are relevant, the recall will be high but not the precision. The F-score is important to get an overall overview of the capacity of the system. The most common F-score is the $F_1$. It corresponds to the harmonic mean of the precision and recall. It is computed as follows.
$$ F_1 = 2 \cdot \frac{recall \cdot prescision}{recall + precision} $$
In this case, both precision and recall are as important as each other. It is to be noted that a generalization of the $F_1$ exists, the $F_\beta$ socre where you can give more or less importance to the recall or the precision. It can be useful in some use cases where one is more important than the other.

Here is a figure extracted from Wikipedia that summarize how recall and precision are computed.
![precision and recall](https://upload.wikimedia.org/wikipedia/commons/thumb/2/26/Precisionrecall.svg/264px-Precisionrecall.svg.png)

## c) Information Retrieval Models

An IR model can mathematically represents the entities at play in an information retrieval task. That is the user need (a query), and how relevant documents are selected and ordered. The first model to have been studied is the **Boolean Model**. In this model, a document *d* is represented as a set of terms $t_i$. Imagine for example a collection $C$ composed of a set of documents as follow:

$d_1 = (t_1, t_4, t_5)$ &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; $d_2 = (t_2, t_3)$ &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; $d_3 = (t_1, t_3, t_4, t_6)$

A boolean query can then be expressed using the terms $t_i$ and a list of boolean opeartors: AND ($\land$), OR ($\lor$) and NOT ($\neg$). For example, the following query will match documents $d_2$ and $d_3$.

$q = (t_1 \lor t_2) \land t_3$

This model is quite simple but has many drawbacks. First, it is not always easy for users to express boolean queries, which can sometimes be quite complex. Moreover, there is no relevancy order in the resulting documents: either the document match the query or it does not. The Boolean Model can still be useful in some cases but it is still too coarse, especially when dealing with unstructured textual documents.



The next model we will study is the **Vector Space Model**. Its basic idea is to represent documents as a vector of terms in the vecor space generated by the list of terms $t_i$ in the whole collection (where one term corresponds to one dimension in this space). For example, in the collection $C$ defined previously, there is a total of $6$ terms, thus generating a vector space of $6$ dimensions. The terms in every documents and every queries are associated with a weight $w$ representing their importance. This weight is computed using statistics methods, the most common one being TF-IDF. The Vector Space Model is considered as a Bag-of-Words model, because the order of terms in the documents does not matter. Using this representation, the document "the cat eats the mouse" has the exact same representation as the document "the mouse eats the cat". This can appear as a huge drawback but in many cases, it is often enough to extract relevant documents from a collection. This flaw can be alleviated using different analysis methods like n-grams for example. To sum-up, the previously defined collection $C$ can be represented as the follwing matrix.
$$ \left( \begin{matrix} w_{1,1} & w_{2,1} & w_{3,1} & w_{4,1} & w_{5,1} & w_{6,1} \\ w_{1,2} & w_{2,2} & w_{3,2} & w_{4,2} & w_{5,2} & w_{6,2} \\ w_{1,3} & w_{2,3} & w_{3,3} & w_{4,3} & w_{5,3} & w_{6,3} \end{matrix} \right) $$

Each row in this matrix corresponds to a document in the collection. The weights $w_{i,j}$ can be computed using TF-IDF. 

The idea behing TF-IDF is to associate a weight with each term according to its importance in the document. Two apsects are considered to evaluate the importance of a term:
- **Term frequency (TF)**: the more a term appear in a document, the more important it is. There are several ways of computing this ([TF computations](https://en.wikipedia.org/wiki/Tf%E2%80%93idf#Term_frequency_2)). The most intuitive one is just to count the number of times a term $t_i$ appear in a document. Other computation methods exist, for example to normalize this frequency against the length of the document. However, we can't just take into account term frequency as a measure of importance of a word because a lot of terms are very common but do not convey much information (articles and pronouns are often quite frequent in every document for example). One way of alleviating this is to consider how rare a term is in the entire collection, this is the inverse document frequency.
- **Inverse document frequency (IDF)**: in most languages, most of the information is conveyed by words that are quite rare. The IDF measures how rare are these words in a collection, which is equivalent to how much information is conveyed by a term. For example, if the article 'the' is very frequent in a particular document (high TF), it is also very frequent in the entire collection (low IDF). Its TF-IDF weight will thus be quite low, which is a good thing because it is indeed not very important. On the contrary, if a term appear in a document, but does not appear in any other document of the collection, the IDF will be high. See [this page](https://en.wikipedia.org/wiki/Tf%E2%80%93idf#Inverse_document_frequency_2) for various ways of computing the IDF.

The final weight is computed by multiplying the TF and the IDF.

A query defined by the user will also be represented as a vector in the same vector space. The relevance of every documents towards the query can then be computed using a vector similarity function (the most common one being the cosinus distance, but other exist).